In [1]:
from pyscf import gto, scf
import numpy as np
import pyqmc.slater as slater
from extract import pyscf_parameter
from mc import np_initial_guess
from determinant_tools import single_occupation_list

mol = gto.Mole()

mol.atom = '''
O 0.000000 0.000000 0.117790
H 0.000000 0.755453 -0.471161
H 0.000000 -0.755453 -0.471161
'''
mol.basis = 'sto-3g'

mol.build()
parameters = pyscf_parameter(mol)
configs = np_initial_guess(mol, 1000)

converged SCF energy = -74.963146775618


In [3]:
import sys
print(sys.version)

3.9.20 (main, Oct  3 2024, 02:24:59) 
[Clang 14.0.6 ]


In [2]:
mf = scf.RHF(mol)
mf.kernel()

converged SCF energy = -74.9631467756179


-74.96314677561793

In [9]:
import functools
import pyscf.pbc.gto.eval_gto

In [8]:
mf.mo_occ

array([2., 2., 2., 2., 2., 0., 0.])

In [30]:

import numpy as np
import pyqmc.gpu as gpu
import pyqmc.determinant_tools as determinant_tools
import pyqmc.orbitals
import pyqmc.pyscftools
import warnings

In [4]:
# MIT License
#
# Copyright (c) 2019-2024 The PyQMC Developers
#
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.

import pyscf
import pyscf.pbc
import pyscf.mcscf
import pyscf.fci
import h5py
import json
import numpy as np
import pyqmc.determinant_tools as determinant_tools
import pyqmc.orbitals as orbitals
import pyqmc.supercell as supercell
import pyqmc.twists as twists


def recover_pyscf(chkfile, ci_checkfile=None, cancel_outputs=True):
    """Generate pyscf objects from a pyscf checkfile, in a way that is easy to use for pyqmc. The chkfile should be saved by setting mf.chkfile in a pyscf SCF object.

    It is recommended to write and recover the objects, rather than trying to use pyscf objects directly when dask parallelization is being used, since by default the pyscf objects contain unserializable objects. (this may be changed in the future)

    `cancel_outputs` will set the outputs of the objects to None. You may need to set `cancel_outputs=False` if you are using this to input to other pyscf functions.

    Typical usage::

        mol, mf = recover_pyscf("dft.hdf5")

    :param chkfile: The filename to read from.
    :type chkfile: string
    :return: mol, mf
    :rtype: pyscf Mole, SCF objects"""

    with h5py.File(chkfile, "r") as f:
        periodic = "a" in json.loads(f["mol"][()]).keys()

    if not periodic:
        mol = pyscf.lib.chkfile.load_mol(chkfile)
        with h5py.File(chkfile, "r") as f:
            mo_occ_shape = f["scf/mo_occ"].shape
        if cancel_outputs:
            mol.output = None
            mol.stdout = None
        if len(mo_occ_shape) == 2:
            mf = pyscf.scf.UHF(mol)
        elif len(mo_occ_shape) == 1:
            mf = pyscf.scf.ROHF(mol) if mol.spin != 0 else pyscf.scf.RHF(mol)
        else:
            raise Exception("Couldn't determine type from chkfile")
    else:
        mol = pyscf.pbc.lib.chkfile.load_cell(chkfile)
        with h5py.File(chkfile, "r") as f:
            has_kpts = "mo_occ__from_list__" in f["/scf"].keys()
            if has_kpts:
                rhf = "000000" in f["/scf/mo_occ__from_list__/"].keys()
            else:
                rhf = len(f["/scf/mo_occ"].shape) == 1
        if cancel_outputs:
            mol.output = None
            mol.stdout = None
        if not rhf and has_kpts:
            mf = pyscf.pbc.scf.KUHF(mol)
        elif has_kpts:
            mf = pyscf.pbc.scf.KROHF(mol) if mol.spin != 0 else pyscf.pbc.scf.KRHF(mol)
        elif rhf:
            mf = pyscf.pbc.scf.ROHF(mol) if mol.spin != 0 else pyscf.pbc.scf.RHF(mol)
        else:
            mf = pyscf.pbc.scf.UHF(mol)
    mf.__dict__.update(pyscf.scf.chkfile.load(chkfile, "scf"))

    if ci_checkfile is not None:
        casdict = pyscf.lib.chkfile.load(ci_checkfile, "ci")
        if casdict is None:
            casdict = pyscf.lib.chkfile.load(ci_checkfile, "mcscf")
        with h5py.File(ci_checkfile, "r") as f:
            hci = "ci/_strs" in f.keys()
        if hci:
            mc = pyscf.hci.SCI(mol)
        else:
            if len(casdict["mo_coeff"].shape) == 3:
                mc = pyscf.mcscf.UCASCI(mol, casdict["ncas"], casdict["nelecas"])
            else:
                mc = pyscf.mcscf.CASCI(mol, casdict["ncas"], casdict["nelecas"])
        mc.__dict__.update(casdict)

        return mol, mf, mc
    return mol, mf


def orbital_evaluator_from_pyscf(
    mol, mf, mc=None, twist=0, determinants=None, tol=None, eval_gto_precision=None, evaluate_orbitals_with="pyscf",
):
    """
    mol: A Mole object
    mf: a pyscf mean-field object
    mc: a pyscf multiconfigurational object. Supports HCI and CAS
    twist: the twist of the calculation (units?)
    determinants: A list of determinants suitable to pass into create_packed_objects
    tol: smallest determinant weight to include in the wave function.
    eval_gto_precision: desired value of orbital at rcut, used for determining rcut for periodic system. Default value = 0.01

    You cannot pass both mc/tol and determinants.

    :returns:
        * detwt: array of weights for each determinant
        * occup: which orbitals go in which determinants
        * map_dets: given a determinant in detwt, which determinant in occup it corresponds to
        * an orbital evaluator chosen based on the inputs.
    """

    periodic = hasattr(mol, "a")
    f_max_orb = lambda a: int(np.max(a, initial=0)) + 1 if len(a) > 0 else 0

    if periodic:
        mf = pyscf.pbc.scf.addons.convert_to_khf(mf)

    try:
        mf = mf.to_uhf()
    except TypeError:
        mf = mf.to_uhf(mf)

    if determinants is None:
        determinants = determinants_from_pyscf(mol, mf, mc=mc, tol=tol)

    if hasattr(mc, "mo_coeff"):
        # assume no kpts for mc calculation
        _mo_coeff = mc.mo_coeff
        if len(_mo_coeff.shape) == 2:  # restricted spin: create up and down copies
            _mo_coeff = [_mo_coeff, _mo_coeff]
        if periodic:
            _mo_coeff = [m[np.newaxis] for m in _mo_coeff]  # add kpt dimension
    else:
        _mo_coeff = mf.mo_coeff

    if periodic:
        if not hasattr(mol, "original_cell"):
            mol = supercell.get_supercell(mol, np.eye(3))
        kinds = twists.create_supercell_twists(mol, mf)["primitive_ks"][twist]
        if len(kinds) != mol.scale:
            raise ValueError(
                f"Found {len(kinds)} k-points but should have found {mol.scale}."
            )
        kpts = mf.kpts[kinds]

        max_orb = [[[f_max_orb(k) for k in s] for s in det] for wt, det in determinants]
        max_orb = np.amax(max_orb, axis=0)
        mo_coeff = [
            [_mo_coeff[s][k][:, 0 : max_orb[s][k]] for k in kinds] for s in [0, 1]
        ]

        evaluator = orbitals.PBCOrbitalEvaluatorKpoints(
            mol, mo_coeff, kpts, eval_gto_precision, evaluate_orbitals_with
        )
        determinants = determinant_tools.flatten_determinants(
            determinants, max_orb, kinds
        )
    else:
        max_orb = [[f_max_orb(s) for s in det] for wt, det in determinants]
        max_orb = np.amax(max_orb, axis=0)
        mo_coeff = [_mo_coeff[spin][:, 0 : max_orb[spin]] for spin in [0, 1]]
        evaluator = orbitals.MoleculeOrbitalEvaluator(mol, mo_coeff, evaluate_orbitals_with)

    detcoeff, occup, det_map = determinant_tools.create_packed_objects(
        determinants, tol=tol
    )
    return detcoeff, occup, det_map, evaluator


def determinants_from_pyscf(mol, mf, mc=None, tol=-1):
    periodic = hasattr(mol, "a")
    if mc is None:
        determinants = single_determinant_from_mf(mf)
    elif periodic:
        determinants = pbc_determinants_from_casci(mc, cutoff=tol)

    if mc is not None and not periodic:
        determinants = interpret_ci(mc, tol)
    return determinants


def single_determinant_from_mf(mf, weight=1.0):
    """
    Creates a determinant list for a single determinant from SCF object
    """
    try:
        mf = mf.to_uhf()
    except TypeError:
        mf = mf.to_uhf(mf)
    if hasattr(mf, "kpts"):
        occupation = [[list(np.nonzero(k > 0.5)[0]) for k in s] for s in mf.mo_occ]
    else:
        occupation = [list(np.nonzero(s > 0.5)[0]) for s in mf.mo_occ]
    return [(weight, occupation)]


def pbc_determinants_from_casci(mc, orbitals=None, cutoff=0.05):
    if hasattr(mc.ncore, "__len__"):
        nocc = [c + e for c, e in zip(mc.ncore, mc.nelecas)]
    else:
        nocc = [mc.ncore + e for e in mc.nelecas]
    if orbitals is None:
        orbitals = np.arange(mc.ncore, mc.ncore + mc.ncas)
    if not hasattr(orbitals[0], "__len__"):
        orbitals = [orbitals, orbitals]
    deters = pyscf.fci.addons.large_ci(mc.ci, mc.ncas, mc.nelecas, tol=-1)
    determinants = []
    for x in deters:
        if abs(x[0]) > cutoff:
            allorbs = [
                [translate_occ(x[1], orbitals[0], nocc[0])],
                [translate_occ(x[2], orbitals[1], nocc[1])],
            ]
            determinants.append((x[0], allorbs))
    return determinants


def translate_occ(x, orbitals, nocc):
    a = determinant_tools.binary_to_occ(x, 0)[0]
    orbitals_without_active = list(range(nocc))
    for o in orbitals:
        if o in orbitals_without_active:
            orbitals_without_active.remove(o)
    return orbitals_without_active + [orbitals[i] for i in a]


def interpret_ci(mc, tol):
    """
    Copies over determinant coefficients and MO occupations
    for a multi-configuration calculation mc.

    returns:
    detwt: array of weights for each determinant
    occup: which orbitals go in which determinants
    map_dets: given a determinant in detwt, which determinant in occup it corresponds to
    """
    ncore = mc.ncore if hasattr(mc, "ncore") else 0
    # find multi slater determinant occupation
    if hasattr(mc, "_strs"):  # if this is a HCI object, it will have _strs
        deters = deters_from_hci(mc, tol)
    else:
        deters = pyscf.fci.addons.large_ci(mc.ci, mc.ncas, mc.nelecas, tol=-1)
    return determinant_tools.reformat_binary_dets(deters, ncore=ncore, tol=tol)


def deters_from_hci(mc, tol):
    bigcis = np.abs(mc.ci) > tol
    nstrs = int(mc._strs.shape[1] / 2)
    deters = []
    # In pyscf, the first n/2 strings represent the up determinant and the second
    # represent the down determinant.
    for c, s in zip(mc.ci[bigcis], mc._strs[bigcis, :]):
        s1 = "".join(str(bin(p)).replace("0b", "") for p in s[0:nstrs])
        s2 = "".join(str(bin(p)).replace("0b", "") for p in s[nstrs:])
        deters.append((c, s1, s2))
    return deters


In [25]:
from pyscf import gto, scf, mcscf
import pyqmc

# 예시 분자 객체 생성
mol = gto.M(atom="H 0 0 0; H 0 0 0.74;", basis="sto-3g")

# Mean-field 객체 생성
mf = scf.RHF(mol)
mf.kernel()

import pyqmc.slater as slater
wf = slater.Slater(mol, mf)

converged SCF energy = -1.11675930739643


In [30]:
wf.aos

AttributeError: 'Slater' object has no attribute 'aos'

In [21]:
import pyqmc.orbitals



AttributeError: module 'pyqmc.orbitals' has no attribute 'aos'

In [ ]:
mol.atom_coords()

array([[ 0.        ,  0.        ,  0.22259084],
       [ 0.        ,  1.42759927, -0.89036525],
       [ 0.        , -1.42759927, -0.89036525]])

In [21]:
print("Molecule:", mol)
print("Mean-Field Object:", mf)


Molecule: <pyscf.gto.mole.Mole object at 0x111694410>
Mean-Field Object: <pyscf.scf.uhf.UHF object at 0x14f716f00>


In [22]:
mf = scf.RHF(mol)
mf.kernel()

import numpy as np

# 필요한 매개변수 설정
twist = 0
determinants = None
tol = 1e-6
evaluate_orbitals_with = "pyscf"

f_max_orb = lambda a: int(np.max(a, initial=0)) + 1 if len(a) > 0 else 0

try:
    mf = mf.to_uhf()
except TypeError:
    mf = mf.to_uhf(mf)

if determinants is None:
    determinants = [(1.0, [list(np.nonzero(s > 0.5)[0]) for s in mf.mo_occ])]

_mo_coeff = mf.mo_coeff

max_orb = [[f_max_orb(s) for s in det] for wt, det in determinants]
max_orb = np.amax(max_orb, axis=0)
mo_coeff = [_mo_coeff[spin][:, 0 : max_orb[spin]] for spin in [0, 1]]

# 오비탈 평가기 생성 (간단한 예제)
class MoleculeOrbitalEvaluator:
    def __init__(self, mol, mo_coeff, evaluate_orbitals_with):
        self.mol = mol
        self.mo_coeff = mo_coeff
        self.evaluate_orbitals_with = evaluate_orbitals_with

evaluator = MoleculeOrbitalEvaluator(mol, mo_coeff, evaluate_orbitals_with)

# 결정자 계수, 점유 상태, 결정자 맵 생성 (간단한 예제)
detcoeff = np.array([wt for wt, _ in determinants])
occup = [det for _, det in determinants]
det_map = {i: i for i in range(len(determinants))}

print("Determinator Coefficients:", detcoeff)
print("Determinant Occupations:", occup)
print("Determinant Map:", det_map)
print("Evaluator:", evaluator)


converged SCF energy = -74.9631467756179
Determinator Coefficients: [1.]
Determinant Occupations: [[[0, 1, 2, 3, 4], [0, 1, 2, 3, 4]]]
Determinant Map: {0: 0}
Evaluator: <__main__.MoleculeOrbitalEvaluator object at 0x12327b320>


In [19]:
mf = mf.to_uhf()
occupation = [[list(np.nonzero(k > 0.5)[0]) for k in s] for s in mf.mo_occ]
occupation = [list(np.nonzero(s > 0.5)[0]) for s in mf.mo_occ]
determinants = [(1.0), occupation]
f_max_orb = lambda a: int(np.max(a, initial=0)) + 1 if len(a) > 0 else 0
max_orb = [[f_max_orb(s) for s in det] for wt, det in determinants]


TypeError: cannot unpack non-iterable float object

In [17]:
mf.mo_coeff.shape

(2, 7, 7)

In [10]:
hasattr(mf, "kpts")

False

In [ ]:
mol.atom_coords()

array([[ 0.        ,  0.        ,  0.22259084],
       [ 0.        ,  1.42759927, -0.89036525],
       [ 0.        , -1.42759927, -0.89036525]])

In [11]:
mf.mo_occ

array([[1., 1., 1., 1., 1., 0., 0.],
       [1., 1., 1., 1., 1., 0., 0.]])

In [96]:
ao_labels = mol.ao_labels()
ao_labels

['0 O 1s    ',
 '0 O 2s    ',
 '0 O 2px   ',
 '0 O 2py   ',
 '0 O 2pz   ',
 '1 H 1s    ',
 '2 H 1s    ']

In [104]:
atom_id = int(ao_labels[5].split()[0])
atom_id

1

In [90]:
mf = scf.RHF(mol)
mf.kernel()

converged SCF energy = -74.963146775618


np.float64(-74.96314677561796)

In [ ]:
wf = slater.Slater(mol, mf, *kwargs)

In [20]:
nconfig = 1000 # the number of configurations

epos = np.zeros((nconfig, np.sum(mol.nelec), 3))
wts = mol.atom_charges()
wts = wts / wts.sum()
print(f"wts is per atom charge rate :{wts}")

for s in [0,1]: # assign spin chareg
    neach = np.array(
        np.floor(mol.nelec[s] * wts), dtype=int
    )
    print(neach)
    nassigned = np.sum(neach)
    totleft = int(mol.nelec[s] - nassigned)
    print(totleft)

wts is per atom charge rate :[0.8 0.1 0.1]
[4 0 0]
1
[4 0 0]
1


In [24]:
s = 0
ind0 = s * mol.nelec[0]
epos[:, ind0:ind0 + nassigned, :]

0

In [27]:
mol.atom_coords()

array([[ 0.        ,  0.        ,  0.22259084],
       [ 0.        ,  1.42759927, -0.89036525],
       [ 0.        , -1.42759927, -0.89036525]])

In [30]:
nassigned = np.sum(neach)
nassigned

np.int64(4)

In [45]:
nex = np.array([4,1,0])

In [46]:
np.repeat(mol.atom_coords(), nex, axis=0)

array([[ 0.        ,  0.        ,  0.22259084],
       [ 0.        ,  0.        ,  0.22259084],
       [ 0.        ,  0.        ,  0.22259084],
       [ 0.        ,  0.        ,  0.22259084],
       [ 0.        ,  1.42759927, -0.89036525]])

In [42]:
mol.atom_coords()

array([[ 0.        ,  0.        ,  0.22259084],
       [ 0.        ,  1.42759927, -0.89036525],
       [ 0.        , -1.42759927, -0.89036525]])

In [48]:
inds = np.argpartition(
    np.random.random((nconfig, len(wts))), totleft, axis=1
)[:, :totleft]

inds

array([[1],
       [2],
       [0],
       [1],
       [0],
       [1],
       [1],
       [2],
       [0],
       [1],
       [2],
       [1],
       [0],
       [2],
       [0],
       [1],
       [0],
       [1],
       [0],
       [2],
       [1],
       [1],
       [2],
       [0],
       [2],
       [1],
       [2],
       [2],
       [0],
       [2],
       [0],
       [2],
       [2],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [2],
       [2],
       [2],
       [2],
       [1],
       [1],
       [2],
       [1],
       [0],
       [2],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [2],
       [2],
       [0],
       [2],
       [2],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [2],
       [2],
       [1],
       [2],
       [1],
       [1],
       [0],
       [1],
       [1],
       [2],
       [1],
    

In [87]:
import numpy as np

# 예제 데이터
nconfig = 5
wts = [0.5, 0.3, 0.2]
totleft = 1

# 무작위 숫자 배열 생성
random_array = np.random.random((nconfig, len(wts)))
print("Random Array:\n", random_array)

# 부분 정렬하여 인덱스 선택
inds = np.argpartition(random_array, totleft, axis=1)[:, :totleft]
print("Selected Indices:\n", inds)

Random Array:
 [[0.64064259 0.51501874 0.38510855]
 [0.82772493 0.2423516  0.94399085]
 [0.10945212 0.15857013 0.39536755]
 [0.50554496 0.46454346 0.57537744]
 [0.50086082 0.05961323 0.85131223]]
Selected Indices:
 [[2]
 [1]
 [0]
 [1]
 [1]]


In [63]:
arr = np.array([7, 2, 5, 3, 8, 4, 6, 1])
indices = np.argpartition(arr, 4)
arr[indices][4:]

array([5, 8, 6, 7])

In [88]:
inds = np.argpartition(
    np.random.random((nconfig, len(wts))), totleft, axis=1
)[:, :totleft]
print(inds)
mol.atom_coords()[inds]

[[2]
 [0]
 [1]
 [1]
 [0]]


array([[[ 0.        , -1.42759927, -0.89036525]],

       [[ 0.        ,  0.        ,  0.22259084]],

       [[ 0.        ,  1.42759927, -0.89036525]],

       [[ 0.        ,  1.42759927, -0.89036525]],

       [[ 0.        ,  0.        ,  0.22259084]]])

In [86]:
mol.atom_coords()

array([[ 0.        ,  0.        ,  0.22259084],
       [ 0.        ,  1.42759927, -0.89036525],
       [ 0.        , -1.42759927, -0.89036525]])

In [66]:
mol.atom_coords()

array([[ 0.        ,  0.        ,  0.22259084],
       [ 0.        ,  1.42759927, -0.89036525],
       [ 0.        , -1.42759927, -0.89036525]])